In [0]:
import pandas as pd
import numpy as np
import math
import statistics 
from scipy.stats import shapiro
from datetime import date, timedelta

In [0]:
transfers = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/transfers_trophies.pkl")
gk_stats = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/gk_stats_wide.pkl")
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")
stats_detailed = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/player_stats_wide.pkl")
gk_ratios = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/gk_ratios.pkl")
players_ratios = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/players_ratios.pkl")
transfers.dropna(inplace = True)

In [0]:
def summer_winter(date):
  month = date.month
  if 0 <= month <= 2:
    return "winter"
  elif 6 <= month <= 8:
    return "summer"
  else:
    return "mid_seson"
transfers["type"] = transfers.date.apply(summer_winter)
transfers["type"] = transfers["type"].astype('category') 

In [0]:
# transfers_net = transfers_net[(transfers_net.loan == False) & (transfers_net.fee >0)]
# len(transfers_net.tm_id.unique())

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [0]:
gk_stats["year"] = gk_stats["date"].apply(lambda x : x.year)

In [0]:
print(gk_ratios.columns)

Index(['tm_id', 'season', 'date', 'fmpct', 'fwpct', 'mpson', 'mpsoff',
       'gmpson', 'gmpsoff', 'apps', 'mpcs', 'mpgc', 'gcpcs', 'avgmpgc',
       'avgmpcs', 'cum_fmpct', 'cum_fwpct', 'cum_mpson', 'cum_mpsoff',
       'cum_gmpson', 'cum_gmpsoff', 'cum_apps', 'cum_mpcs', 'cum_mpgc',
       'cum_gcpcs', 'cum_avgmpgc', 'cum_avgmpcs'],
      dtype='object')


In [0]:
gk_model_data = gk_stats[["tm_id","season","year","mp","ppg","cs","gc","cum_cs","cum_gc","cum_ppg"]]

In [0]:
gk_model_ratios = gk_ratios[["tm_id","season","fwpct","gcpcs","cum_mpcs","cum_gcpcs","cum_fwpct","cum_avgmpgc"]]

In [0]:
gk_stats_ratios = pd.merge(gk_model_data,gk_model_ratios, on = ["tm_id","season"])

In [0]:
transfers_detailed = pd.merge(transfers, players[["tm_id","main_field_position","continent","field_position",
                                                  "height","sf","dob"]],on = "tm_id")

In [0]:
def get_age(age_delta):
  result = math.nan
  try:
    result = int(age_delta / timedelta(days=365.2425))
  except:
    pass
  return result    

In [0]:
transfers_detailed["age"] = transfers_detailed["date"] - transfers_detailed["dob"]
transfers_detailed.age = transfers_detailed.age.apply(get_age) 

In [0]:
gk_transfers = transfers_detailed[transfers_detailed.main_field_position == "goalkeepers"]

In [0]:
gk_transfers_mv = pd.merge(gk_transfers, markval[["tm_id","season","mv","cum_mv"]])

In [0]:
gk_transfers_mv.sf = gk_transfers_mv.sf.astype("object") 
gk_transfers_mv.loc[gk_transfers_mv.sf == "no_info", "sf"] = np.nan 

In [0]:
gk_transfers_info = pd.merge(gk_transfers_mv,gk_stats_ratios, on = ["tm_id","year"])

In [0]:
gk_transfers_info.rename({"season_x":"transfer_season", "season_y":"stats_season"}, axis = "columns", inplace = True)

In [0]:
gk_transfers_info.info()
gk_transfers_info.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/gk_transfers_info.pkl")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 479
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 480 non-null    datetime64[ns]
 1   tm_id                480 non-null    int64         
 2   from                 480 non-null    object        
 3   to                   480 non-null    object        
 4   fee                  480 non-null    float64       
 5   mv                   480 non-null    float64       
 6   transfer_season      480 non-null    object        
 7   loan                 480 non-null    bool          
 8   year                 480 non-null    float64       
 9   runner_up            480 non-null    float64       
 10  winner               480 non-null    float64       
 11  cum_runner_up        480 non-null    float64       
 12  cum_winner           480 non-null    float64       
 13  type                 480 non-null  

In [0]:
players_transfers = transfers_detailed[transfers_detailed.main_field_position != "goalkeepers"]
players_stats_ratios = pd.merge(stats_detailed, players_ratios, on = ["tm_id","season"])
players_transfers_mv = pd.merge(players_transfers, markval[["tm_id","season","mv","cum_mv"]])

In [0]:
players_transfers_mv = pd.merge(players_transfers_mv,players_stats_ratios, on = ["tm_id","year"])

In [0]:
players_transfers_mv = players_transfers_mv[players_transfers_mv.age.notnull()]

In [0]:
players_transfers_mv.drop(["date_x","date_y"], axis = "columns", inplace = True)
players_transfers_mv.rename({"season_x":"transfer_season","season_y":"stats_season"}, inplace = True, axis = "columns")
players_transfers_mv

,date,tm_id,from,to,fee,mv,transfer_season,loan,year,runner_up,winner,cum_runner_up,cum_winner,type,main_field_position,continent,field_position,height,sf,dob,age,cum_mv,stats_season,app,a,g,mp,og,pg,rc,syc,s,soff,son,yc,mpg,ppg,cum_app,cum_mp,cum_og,cum_rc,cum_syc,cum_s,cum_soff,cum_son,cum_yc,cum_a,cum_g,cum_mpg,cum_pg,cum_ppg,cum_games_won,fmpct,fwpct,mpson,mpsoff,apps,gppg,gpg,mpa,apg,mpyc,mprc,mpsyc,ycprc,cum_fmpct,cum_fwpct,cum_mpson,cum_mpsoff,cum_apps,cum_gppg,cum_gpg,cum_mpa,cum_apg,cum_mpyc,cum_mprc,cum_mpsyc,cum_ycprc
0,2019-07-01,281963,Atlético Madrid,Bayern Munich,80000000.0,70000000.0,19/20,False,2019.0,2.0,1.0,3.0,5.0,summer,defenders,EU,CB,182.0,left,1996-02-14,23.0,52500000.0,18/19,22.0,0.0,1.0,1770.0,0.0,0.0,0.0,0.0,24.0,4.0,2.0,7.0,279.50,2.3500,269.0,17410.0,0.0,0.0,0.0,330.0,75.0,81.0,25.0,45.0,23.0,875.000000,1.0,1.235000,110.738333,0.768229,0.601780,885.000000,442.500000,1.090909,NaN,2.911458,NaN,NaN,252.857143,NaN,NaN,NaN,0.687760,0.282769,329.625000,201.270833,1.057453,NaN,9.114583,NaN,NaN,322.833333,NaN,NaN,NaN
1,2017-07-01,166601,TSG Hoffenheim,Bayern Munich,20000000.0,25000000.0,17/18,False,2017.0,0.0,1.0,2.0,0.0,summer,defenders,EU,CB,195.0,right,1995-09-03,21.0,17500000.0,16/17,34.0,3.0,2.0,3007.0,1.0,0.0,0.0,0.0,34.0,2.0,0.0,4.0,722.00,0.9250,117.0,9896.0,0.0,0.0,3.0,140.0,6.0,7.0,21.0,2.0,0.0,486.166667,0.0,1.074167,41.892500,0.921262,0.284056,NaN,1503.500000,1.000000,NaN,7.520833,1002.333333,10.440972,751.750000,NaN,NaN,NaN,0.899774,0.326381,NaN,1966.666667,1.017857,NaN,5.064236,NaN,NaN,294.672222,NaN,NaN,NaN
2,2007-08-21,26485,Hertha BSC,Hamburger SV,1100000.0,750000.0,07/08,False,2007.0,0.0,0.0,0.0,0.0,summer,defenders,EU,CB,192.0,right,1988-09-03,18.0,425000.0,06/07,26.0,0.0,0.0,1958.0,0.0,0.0,0.0,0.0,28.0,5.0,4.0,6.0,0.00,0.6000,159.0,10782.0,0.0,0.0,0.0,165.0,51.0,41.0,4.0,28.0,11.0,263.250000,0.0,1.930000,102.290000,0.728423,0.145685,489.500000,391.600000,1.076923,NaN,0.000000,NaN,NaN,326.333333,NaN,NaN,NaN,0.762813,0.489160,440.708333,268.422619,1.010638,NaN,2.742188,350.065657,3.646517,1159.833333,NaN,NaN,NaN
3,2007-08-21,26485,Hertha BSC,Hamburger SV,1100000.0,750000.0,07/08,False,2007.0,0.0,0.0,0.0,0.0,summer,defenders,EU,CB,192.0,right,1988-09-03,18.0,750000.0,06/07,26.0,0.0,0.0,1958.0,0.0,0.0,0.0,0.0,28.0,5.0,4.0,6.0,0.00,0.6000,159.0,10782.0,0.0,0.0,0.0,165.0,51.0,41.0,4.0,28.0,11.0,263.250000,0.0,1.930000,102.290000,0.728423,0.145685,489.500000,391.600000,1.076923,NaN,0.000000,NaN,NaN,326.333333,NaN,NaN,NaN,0.762813,0.489160,440.708333,268.422619,1.010638,NaN,2.742188,350.065657,3.646517,1159.833333,NaN,NaN,NaN
4,2010-07-01,26485,Hamburger SV,Man City,12500000.0,15000000.0,10/11,False,2010.0,0.0,0.0,0.0,0.0,summer,defenders,EU,CB,192.0,right,1988-09-03,21.0,9500000.0,09/10,41.0,4.0,2.0,3197.0,1.0,0.0,1.0,0.0,44.0,10.0,3.0,7.0,223.75,2.2800,277.0,19441.0,0.0,0.0,0.0,286.0,80.0,68.0,9.0,57.0,22.0,216.033333,0.0,1.625333,150.072444,0.756866,0.575218,1065.666667,319.700000,1.073171,NaN,2.330729,799.250000,8.325521,456.714286,3197.0,NaN,7.0,0.693990,0.373558,318.000000,206.489583,1.036902,NaN,2.250347,343.187500,3.574870,2745.500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7336,2011-07-22,44675,FC Barcelona,AS Roma,12000000.0,15000000.0,11/12,False,2011.0,1.0,4.0,0.0,7.0,summer,attackers,EU,ST,173.0,right,1990-08-28,20.0,12500000.0,10/11,37.0,3.0,7.0,1558.0,0.0,1.0,0.0,0.0,51.0,10.0,20.0,2.0,143.50,2.1050,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.318219,0.223284,77.900000,155.800000,1.378378,7.0,1.494792,519.333333,5.409722,779.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
7337,2013-07-01,44675,AS Roma,FC Barcelona,13000000.0,8000000.

In [0]:
defenders = players_transfers_mv[players_transfers_mv.main_field_position == "defenders"]
midfielders = players_transfers_mv[players_transfers_mv.main_field_position == "midfielders"]
attackers = players_transfers_mv[players_transfers_mv.main_field_position == "attackers"]

In [0]:
print(defenders.columns)

Index(['date', 'tm_id', 'from', 'to', 'fee', 'mv', 'transfer_season', 'loan',
       'year', 'runner_up', 'winner', 'cum_runner_up', 'cum_winner', 'type',
       'main_field_position', 'continent', 'field_position', 'height', 'sf',
       'dob', 'age', 'cum_mv', 'stats_season', 'app', 'a', 'g', 'mp', 'og',
       'pg', 'rc', 'syc', 's', 'soff', 'son', 'yc', 'mpg', 'ppg', 'cum_app',
       'cum_mp', 'cum_og', 'cum_rc', 'cum_syc', 'cum_s', 'cum_soff', 'cum_son',
       'cum_yc', 'cum_a', 'cum_g', 'cum_mpg', 'cum_pg', 'cum_ppg',
       'cum_games_won', 'fmpct', 'fwpct', 'mpson', 'mpsoff', 'apps', 'gppg',
       'gpg', 'mpa', 'apg', 'mpyc', 'mprc', 'mpsyc', 'ycprc', 'cum_fmpct',
       'cum_fwpct', 'cum_mpson', 'cum_mpsoff', 'cum_apps', 'cum_gppg',
       'cum_gpg', 'cum_mpa', 'cum_apg', 'cum_mpyc', 'cum_mprc', 'cum_mpsyc',
       'cum_ycprc'],
      dtype='object')


In [0]:
defenders_model_data = defenders[["tm_id","age","mv","year","transfer_season",'stats_season',"type","cum_mv","fee","main_field_position","field_position",
                       "a","g","height","sf","app","mp","ppg","yc","rc","fmpct","fwpct","mpyc","cum_games_won",
                       "cum_a","cum_g","cum_yc","cum_rc","cum_mpyc","cum_fmpct","cum_fwpct","runner_up","winner","cum_runner_up","cum_winner"]]

In [0]:
defenders_model_data.info()
defenders_model_data.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/defenders_transfers_info.pkl")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2199 entries, 0 to 7329
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   tm_id                2199 non-null   int64   
 1   age                  2199 non-null   float64 
 2   mv                   2199 non-null   float64 
 3   year                 2199 non-null   float64 
 4   transfer_season      2199 non-null   object  
 5   stats_season         2199 non-null   category
 6   type                 2199 non-null   category
 7   cum_mv               2199 non-null   float64 
 8   fee                  2199 non-null   float64 
 9   main_field_position  2199 non-null   category
 10  field_position       2199 non-null   category
 11  a                    2199 non-null   float64 
 12  g                    2199 non-null   float64 
 13  height               2199 non-null   float64 
 14  sf                   2199 non-null   category
 15  app                  

In [0]:
midfielders_model_data = midfielders[["tm_id","age","mv","year","transfer_season",'stats_season',"type","fee","cum_mv","main_field_position",
                                      "field_position","a","g","height","sf","app","mp","mpg","mpa","ppg","yc","rc","fmpct","fwpct","mpyc",
                                      "cum_games_won","cum_a","cum_g","cum_yc","cum_rc","cum_fmpct","cum_mpg","cum_fwpct","runner_up",
                                      "winner","cum_runner_up","cum_winner"]]

In [0]:
midfielders_model_data.info()
midfielders_model_data.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/midfielders_transfers_info.pkl")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236 entries, 9 to 7340
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   tm_id                2236 non-null   int64   
 1   age                  2236 non-null   float64 
 2   mv                   2236 non-null   float64 
 3   year                 2236 non-null   float64 
 4   transfer_season      2236 non-null   object  
 5   stats_season         2236 non-null   category
 6   type                 2236 non-null   category
 7   fee                  2236 non-null   float64 
 8   cum_mv               2236 non-null   float64 
 9   main_field_position  2236 non-null   category
 10  field_position       2236 non-null   category
 11  a                    2236 non-null   float64 
 12  g                    2236 non-null   float64 
 13  height               2236 non-null   float64 
 14  sf                   2236 non-null   category
 15  app                  

In [0]:
attackers_model_data = attackers[["tm_id","year","age","mv","transfer_season",'stats_season',"type","fee","cum_mv","main_field_position","field_position",
                       "a","g","pg","height","sf","app","mp","mpg","mpa","ppg","gppg","fmpct","fwpct","mpyc","cum_games_won",
                       "cum_a","cum_g","cum_fmpct","cum_mpg","cum_fwpct","runner_up","winner","cum_runner_up","cum_winner"]]

In [0]:
attackers_model_data["gppg"] = attackers_model_data["pg"] / attackers_model_data["g"]
attackers_model_data["cum_gppg"] = attackers_model_data.groupby("tm_id").apply(lambda df : df["gppg"].rolling(2).mean()).values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
attackers_model_data["cum_gppg"].fillna(0, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
attackers_model_data.info()
attackers_model_data.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/attackers_transfers_info.pkl")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2872 entries, 14 to 7338
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   tm_id                2872 non-null   int64   
 1   year                 2872 non-null   float64 
 2   age                  2872 non-null   float64 
 3   mv                   2872 non-null   float64 
 4   transfer_season      2872 non-null   object  
 5   stats_season         2872 non-null   category
 6   type                 2872 non-null   category
 7   fee                  2872 non-null   float64 
 8   cum_mv               2872 non-null   float64 
 9   main_field_position  2872 non-null   category
 10  field_position       2872 non-null   category
 11  a                    2872 non-null   float64 
 12  g                    2872 non-null   float64 
 13  pg                   2872 non-null   float64 
 14  height               2872 non-null   float64 
 15  sf                  

In [0]:
#for all players, mv,age,app,mp,ppg same vars in cumulative
#for gk, cs, gc, same vars in cum 
#for def, a, g, yc, cum_games_won
#for mid, a,g,soff
#for att, a,g,soff

In [0]:
#ratios
#for all players fmpct
#for gk mpcs, gcpcs, fmpct, cum  for same vars
#for def mprc,
#for mid, gpg, mpa, mprc, 
#for att, gppg, 

In [0]:
#fixes fwpct, apps